In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import inception_resnet_v2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.utils import img_to_array, array_to_img
from google.colab.patches import cv2_imshow
import os
import zipfile
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import ast
import glob

In [ ]:
base_path='/content/drive/MyDrive/Dataset/Dementia_sample/Real_data'

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/Dataset/ADNI_tabular/ADNI_subjects.xlsx')
df

,PHASE,RID,PTID,EXAMDATE,Gender,Birthdate,Age,FINAL_DX,AD severity,Certain or Not,CDR,MMSCORE,Bilateral hippocampus,D_subtract,APGEN1,APGEN2,APOE carrier
0,ADNI2,5040,070_S_5040,2013-07-25,2,1936-12-01,77,normal cognition-NC,NaN,NaN,0.0,30,0.005795,0,3,3,0
1,ADNI2,295,002_S_0295,2011-06-02,1,1921-06-01,90,normal cognition-NC,NaN,NaN,0.0,28,0.004041,0,3,4,1
2,ADNI2,295,002_S_0295,2012-05-10,1,1921-06-01,91,normal cognition-NC,NaN,NaN,0.0,22,0.003914,0,3,4,1
3,ADNI1,413,002_S_0413,2006-05-19,2,1929-12-01,77,normal cognition,NaN,4.0,0.0,29,0.004400,0,3,3,0
4,ADNI1,413,002_S_0413,2006-11-15,2,1929-12-01,77,normal cognition,NaN,4.0,0.0,29,0.004296,0,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5187,ADNI3,6345,941_S_6345,2018-05-10,1,1939-01-01,79,MCI,NaN,NaN,0.5,28,0.003906,0,3,4,1
5188,ADNI3,6345,941_S_6345,2019-05-29,1,1939-01-01,80,Mild AD,1.0,NaN,1.0,24,0.003759,0,3,4,1
5189,ADNI3,6345,941_S_6345,2020-09-08,1,1939-01-01,82,Mild AD,1.0,NaN,1.0,22,0.003553,0,3,4,1
5190,ADNI3,6854,941_S_6854,2020-02-14,1,1933-11-01,86,Mild AD,1.0,NaN,0.5,30,0.003654,0,3,4,1


In [ ]:
df = df.drop(['PHASE','RID','AD severity','Certain or Not','CDR'], axis=1)
df

,PTID,EXAMDATE,Gender,Birthdate,Age,FINAL_DX,MMSCORE,Bilateral hippocampus,D_subtract,APGEN1,APGEN2,APOE carrier
0,070_S_5040,2013-07-25,2,1936-12-01,77,normal cognition-NC,30,0.005795,0,3,3,0
1,002_S_0295,2011-06-02,1,1921-06-01,90,normal cognition-NC,28,0.004041,0,3,4,1
2,002_S_0295,2012-05-10,1,1921-06-01,91,normal cognition-NC,22,0.003914,0,3,4,1
3,002_S_0413,2006-05-19,2,1929-12-01,77,normal cognition,29,0.004400,0,3,3,0
4,002_S_0413,2006-11-15,2,1929-12-01,77,normal cognition,29,0.004296,0,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5187,941_S_6345,2018-05-10,1,1939-01-01,79,MCI,28,0.003906,0,3,4,1
5188,941_S_6345,2019-05-29,1,1939-01-01,80,Mild AD,24,0.003759,0,3,4,1
5189,941_S_6345,2020-09-08,1,1939-01-01,82,Mild AD,22,0.003553,0,3,4,1
5190,941_S_6854,2020-02-14,1,1933-11-01,86,Mild AD,30,0.003654,0,3,4,1


In [ ]:
df[(df['PTID'] == '002_S_0295') & (df['EXAMDATE'] == '20110602')]

,PTID,EXAMDATE,Gender,Birthdate,Age,FINAL_DX,MMSCORE,Bilateral hippocampus,D_subtract,APGEN1,APGEN2,APOE carrier
1,002_S_0295,2011-06-02,1,1921-06-01,90,normal cognition-NC,28,0.004041,0,3,4,1


In [ ]:
# #알집 해제
# def open_zip(Data_path):
#     os.chdir(Data_path)
#     zip_files = os.listdir(Data_path)
#     zip_files=[files for files in zip_files if '.zip' in files]
#     for zip_file in zip_files:
#         zipfile.ZipFile(os.path.join(Data_path,zip_file)).extractall()
#     return os.listdir(Data_path)
# data_listdir=open_zip(base_path)

In [ ]:

def fold_path(Data_path):
    fold = collections.defaultdict(list)
    os.chdir(Data_path)
    Data_list = os.listdir(Data_path)
    folder_list = [folder for folder in Data_list if os.path.isdir(os.path.join(Data_path, folder))]
    print(folder_list)
    for folder in folder_list:
        # print(folder)
        files_in_folder=os.listdir(os.path.join(Data_path,folder))
        # print(files_in_folder)
        fold[folder] = files_in_folder
        # print(fold)
    return fold

fold_data = fold_path(base_path)


['Normal', 'Mild AD']


In [ ]:
fold_data.keys()

dict_keys(['Normal', 'Mild AD'])

In [ ]:
data_list = []
data_dict = {}
for class_label, subject_folds_list in fold_data.items():
    for subject_folder in subject_folds_list:
        parts = subject_folder.split('_')
        #dtype str, str
        subject_id, examdate = '_'.join(parts[0:3]), '20'+parts[3]
        ########examdate에 중간바('-') 를 join 안 시켰는데 인식함.
        result_row = df[(df['PTID'] == subject_id) & (df['EXAMDATE'] == examdate)]
        # print(result_row['Gender'].values.shape)#(1,)
        # print(result_row['Gender'].values[0].shape)#() value값으로 받기 위해  dim 제거
        # if not result_row.empty:
        #     print(f"Class: {class_label}, Subject: {subject_folder}, Gender: {result_row['Gender'].values[0]}, Age: {result_row['Age'].values[0]},")
        image_dir=os.path.join(base_path, class_label,subject_folder)
        image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if filename.endswith('.png')]

            # 데이터를 딕셔너리에 저장
            ###### image_paths -> path, Image_Number 가 list형태로 들어갔는데 고침.
        data_dict = {
            'image_path': image_dir,
            'Gender': result_row['Gender'].values[0],
            'Age': result_row["Age"].values[0],
            'Image_Number': [int(os.path.splitext(os.path.basename(path))[0].split('plane')[-1]) for path in image_paths],
            'label': class_label
        }

            # 리스트에 딕셔너리 추가
        data_list.append(data_dict)
final_df = pd.DataFrame(data_list)


In [ ]:
final_df


,image_path,Gender,Age,Image_Number,label
0,/content/drive/MyDrive/Dataset/Dementia_sample...,1,90,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Normal
1,/content/drive/MyDrive/Dataset/Dementia_sample...,1,91,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Normal
2,/content/drive/MyDrive/Dataset/Dementia_sample...,2,82,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Normal
3,/content/drive/MyDrive/Dataset/Dementia_sample...,2,83,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Normal
4,/content/drive/MyDrive/Dataset/Dementia_sample...,2,83,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Normal
...,...,...,...,...,...
609,/content/drive/MyDrive/Dataset/Dementia_sample...,1,78,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Mild AD
610,/content/drive/MyDrive/Dataset/Dementia_sample...,2,88,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Mild AD
611,/content/drive/MyDrive/Dataset/Dementia_sample...,2,88,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Mild AD
612,/content/drive/MyDrive/Dataset/Dementia_sample...,2,79,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Mild AD


In [ ]:
final_df['image_path'][0]


'/content/drive/MyDrive/Dataset/Dementia_sample/Real_data/Normal/002_S_0295_110602'

In [ ]:
## label2idx
final_df['label'] = final_df['label'].map({
    'Normal':0,
    'Mild AD' :1
})

In [ ]:
bins = [50, 60, 70, 80, 90, 100]
labels = ['50대', '60대', '70대', '80대', '90대']
final_df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

In [ ]:
age_group_mapping = {'50대': 0, '60대': 1, '70대': 2, '80대': 3, '90대': 4}
final_df['Age_Group'] = final_df['Age_Group'].map(age_group_mapping)

In [ ]:
final_df.head()
final_df.to_csv('/content/drive/MyDrive/Dataset/Dementia_sample/data_load_Age_Group.csv',index=False)

In [ ]:
# final_df = pd.read_csv('/content/drive/MyDrive/Dataset/Dementia_sample/data_load_Age_Group.csv')

In [ ]:
final_df

,image_path,Gender,Age,Image_Number,label,Age_Group
0,/content/drive/MyDrive/Dataset/Dementia_sample...,1,90,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,2
1,/content/drive/MyDrive/Dataset/Dementia_sample...,1,91,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,4
2,/content/drive/MyDrive/Dataset/Dementia_sample...,2,82,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,4
3,/content/drive/MyDrive/Dataset/Dementia_sample...,2,83,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,2
4,/content/drive/MyDrive/Dataset/Dementia_sample...,2,83,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,2
...,...,...,...,...,...,...
609,/content/drive/MyDrive/Dataset/Dementia_sample...,1,78,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",1,2
610,/content/drive/MyDrive/Dataset/Dementia_sample...,2,88,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",1,2
611,/content/drive/MyDrive/Dataset/Dementia_sample...,2,88,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",1,2
612,/content/drive/MyDrive/Dataset/Dementia_sample...,2,79,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",1,2


In [ ]:

final_df = final_df.drop('Age', axis=1)
final_df.head()


,image_path,Gender,Image_Number,label,Age_Group
0,/content/drive/MyDrive/Dataset/Dementia_sample...,1,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,2
1,/content/drive/MyDrive/Dataset/Dementia_sample...,1,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,4
2,/content/drive/MyDrive/Dataset/Dementia_sample...,2,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,4
3,/content/drive/MyDrive/Dataset/Dementia_sample...,2,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,2
4,/content/drive/MyDrive/Dataset/Dementia_sample...,2,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",0,2


In [ ]:

final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   image_path    614 non-null    object  
 1   Gender        614 non-null    int64   
 2   Image_Number  614 non-null    object  
 3   label         614 non-null    int64   
 4   Age_Group     614 non-null    category
dtypes: category(1), int64(2), object(2)
memory usage: 20.1+ KB


In [ ]:
final_df['Gender'] = final_df['Gender'].astype('category')
final_df['label'] = final_df['label'].astype('int')
final_df['Age_Group'] = final_df['Age_Group'].astype('category')
final_df['Image_Number'] = final_df['Image_Number'].apply(lambda x: [int(num) for num in x])

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   image_path    614 non-null    object  
 1   Gender        614 non-null    category
 2   Image_Number  614 non-null    object  
 3   label         614 non-null    int64   
 4   Age_Group     614 non-null    category
dtypes: category(2), int64(1), object(2)
memory usage: 16.0+ KB


In [ ]:
use_colab = True
assert use_colab in [True, False]
is_train = True

In [ ]:
inception_res_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(299, 299, 3)
)

219055592/219055592 [==============================] - 1s 0us/step


In [ ]:
############ k fold 화
from sklearn.model_selection import train_test_split

train_data,valid_data, train_labels,valid_labels = \
    train_test_split(final_df[['image_path','Gender','Age_Group','Image_Number']], final_df['label'], test_size=0.2, random_state=42)
train_data,test_data, train_labels,test_labels = \
    train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
"""
train_data, train_labels 0.6
valid_data, valid_labels 0.2
test_data, test_labels 0.2
"""
print(train_data.shape, valid_data.shape, test_data.shape)
print(train_labels.shape, valid_labels.shape, test_labels.shape)


(392, 4) (123, 4) (99, 4)
(392,) (123,) (99,)


In [ ]:
print(train_data['image_path'][0])
# print(train_labels)

/content/drive/MyDrive/Dataset/Dementia_sample/Real_data/Normal/002_S_0295_110602


In [ ]:
train_data.head(10)

,image_path,Gender,Age_Group,Image_Number
57,/content/drive/MyDrive/Dataset/Dementia_sample...,2,2,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
571,/content/drive/MyDrive/Dataset/Dementia_sample...,1,2,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
187,/content/drive/MyDrive/Dataset/Dementia_sample...,2,3,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
435,/content/drive/MyDrive/Dataset/Dementia_sample...,1,2,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
598,/content/drive/MyDrive/Dataset/Dementia_sample...,2,1,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
58,/content/drive/MyDrive/Dataset/Dementia_sample...,2,2,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
354,/content/drive/MyDrive/Dataset/Dementia_sample...,1,3,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
243,/content/drive/MyDrive/Dataset/Dementia_sample...,1,2,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
479,/content/drive/MyDrive/Dataset/Dementia_sample...,2,1,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
33,/content/drive/MyDrive/Dataset/Dementia_sample...,2,2,"[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."


In [ ]:
train_data['image_path'][0]

'/content/drive/MyDrive/Dataset/Dementia_sample/Real_data/Normal/002_S_0295_110602'

In [ ]:
# #Generator
# # 이미지 데이터셋 생성 함수 정의
# def create_dataset(df, labels):
#     for index, row in df.iterrows():
#         # 이미지 폴더 경로
#         image_folder = row['image_path']

#         # 이미지 파일 경로 목록
#         image_paths = get_image_paths(image_folder)

#         # 이미지 번호 목록
#         image_numbers = row['Image_Number']

#         for image_number in image_numbers:
#             # 이미지 파일 경로
#             image_path = os.path.join(image_folder, f"plane{image_number}.png")
#             print(image_path)
#             # 이미지 파일 읽기 및 전처리
#             image = tf.io.read_file(image_path)
#             image = tf.image.decode_png(image, channels=3)  # 예시: PNG 이미지를 디코딩

#             # 레이블 가져오기
#             label = labels[index]
#             # print(type(label))
#             gender = row['Gender']
#             Age_Group = row['Age_Group']

#             # image=tf.convert_to_tensor(image, dtype=tf.float32)
#             # gender=tf.convert_to_tensor(gender, dtype=tf.int32)
#             # Age_Group=tf.convert_to_tensor(Age_Group, dtype=tf.int32)
#             # image_number=tf.convert_to_tensor(image_number, dtype=tf.int32)
#             # label=tf.convert_to_tensor(label, dtype=tf.int32)
#             # 다른 feature들과 함께 yield
#             yield (image, gender, Age_Group, image_number, label)

In [ ]:
# # TensorFlow Dataset generator생성
# train_dataset = tf.data.Dataset.from_generator(
#     create_dataset,
#     output_signature=(
#         tf.TensorSpec(shape=(None, None, 3), dtype=tf.float32),  # 이미지
#         tf.TensorSpec(shape=(), dtype=tf.int32),               # Gender
#         tf.TensorSpec(shape=(), dtype=tf.int32),               # Age_Group
#         tf.TensorSpec(shape=(), dtype=tf.int32),               # Image_Number
#         tf.TensorSpec(shape=(), dtype=tf.int32)                # label
#     ),
#     args=(train_data, train_labels)
# )

# train_dataset = train_dataset.shuffle(buffer_size=len(train_data)).batch(30).repeat()


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
def get_image_paths(image_folder):
    return glob.glob(os.path.join(image_folder, "*.png"))

In [ ]:
def create_dataset_for_folder(data,labels):
    data_list =[]
    data_dict = {}
    for index, row in data.iterrows():
        image_dir = row['image_path']
        # image_paths = get_image_paths(image_dir)
        # print(image_paths) # list_len: 392
        labels = labels
        image_numbers = sorted(row['Image_Number'])
        # print(labels) #(392,)
        for image_number in image_numbers:

            data_dict ={
                'image_path' : str(os.path.join(image_dir, f"plane{image_number}.png")),
                'Gender' : row['Gender'],
                'Age_Group' : row['Age_Group'],
                'image_number' : image_number,
                'label' :labels[index]
            }
            # print(data_dict)
            data_list.append(data_dict)
    return pd.DataFrame(data_list)


In [ ]:
train_dataset = create_dataset_for_folder(train_data,train_labels)
train_dataset

test_dataset = create_dataset_for_folder(test_data,test_labels)
test_dataset

valid_dataset = create_dataset_for_folder(valid_data,valid_labels)
valid_dataset

,image_path,Gender,Age_Group,image_number,label
0,/content/drive/MyDrive/Dataset/Dementia_sample...,1,2,0,1
1,/content/drive/MyDrive/Dataset/Dementia_sample...,1,2,1,1
2,/content/drive/MyDrive/Dataset/Dementia_sample...,1,2,2,1
3,/content/drive/MyDrive/Dataset/Dementia_sample...,1,2,3,1
4,/content/drive/MyDrive/Dataset/Dementia_sample...,1,2,4,1
...,...,...,...,...,...
3685,/content/drive/MyDrive/Dataset/Dementia_sample...,1,3,25,1
3686,/content/drive/MyDrive/Dataset/Dementia_sample...,1,3,26,1
3687,/content/drive/MyDrive/Dataset/Dementia_sample...,1,3,27,1
3688,/content/drive/MyDrive/Dataset/Dementia_sample...,1,3,28,1


In [ ]:
train_dataset.iloc[0]

image_path      /content/drive/MyDrive/Dataset/Dementia_sample...
Gender                                                          2
Age_Group                                                       2
image_number                                                    0
label                                                           0
Name: 0, dtype: object

In [ ]:
train_dataset['image_path']=train_dataset['image_path'].astype(str)
train_dataset.dtypes

image_path      object
Gender           int64
Age_Group        int64
image_number     int64
label            int64
dtype: object

In [ ]:
def preprocess_and_augment(image_path, gender,age_group, image_number,label, training = True):

    image = tf.io.read_file(image_path)
    print(image.shape)
    image = tf.image.decode_jpeg(image, channels=3)
    print(image.shape)
    image = tf.image.convert_image_dtype(image, tf.float32)
    # 이미지 증강
    if training:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_contrast(image, lower=0.8, upper=1.2)

    image = tf.image.resize(image, [299, 299])
    gender = tf.cast(gender, tf.int64)
    age_group = tf.cast(age_group, tf.int64)
    image_number = tf.cast(image_number, tf.int64)
    label = tf.cast(label, np.int64)
    return image, (gender,age_group,image_number), label

In [ ]:
# ### 오류 통째로 넣었을떄
# # 오류발생 코드 1
# train_dataset_tensor = tf.data.Dataset.from_tensor_slices((train_dataset.iloc[:,0:3]))
# # train_dataset_tensor = train_dataset_tensor.map(preprocess_and_augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# train_dataset_tensor = train_dataset_tensor.cache().batch(30)
# train_dataset_tensor

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
train_dataset_tensor = tf.data.Dataset.from_tensor_slices((train_dataset['image_path'].values,train_dataset['Gender'].values,train_dataset['Age_Group'].values, train_dataset['image_number'].values,train_dataset['label'].values))
train_dataset_tensor = train_dataset_tensor.map(preprocess_and_augment,num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset_tensor = train_dataset_tensor.cache().batch(30).prefetch(tf.data.experimental.AUTOTUNE)
train_dataset_tensor

()
(None, None, 3)


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None)), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
valid_dataset_tensor = tf.data.Dataset.from_tensor_slices((valid_dataset['image_path'].values,valid_dataset['Gender'].values,valid_dataset['Age_Group'].values, valid_dataset['image_number'].values,valid_dataset['label'].values))
valid_dataset_tensor = valid_dataset_tensor.map(lambda x1, x2, x3, x4, y: preprocess_and_augment(x1, x2, x3, x4, y, training=False) , num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset_tensor = valid_dataset_tensor.cache().batch(batch_size=30).prefetch(tf.data.experimental.AUTOTUNE)
valid_dataset_tensor

()
(None, None, 3)


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None)), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
test_dataset_tensor = tf.data.Dataset.from_tensor_slices((test_dataset['image_path'].values,test_dataset['Gender'].values,test_dataset['Age_Group'].values, test_dataset['image_number'].values,test_dataset['label'].values))
test_dataset_tensor = test_dataset_tensor.map(lambda x1, x2, x3, x4, y: preprocess_and_augment(x1, x2, x3, x4, y, training=False) , num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset_tensor = test_dataset_tensor.cache().batch(batch_size=30).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset_tensor

()
(None, None, 3)


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None)), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
##### functional api
# inception_res_partial = models.Model(inputs=inception_res_model.input, outputs=inception_res_model.output)

# input = tf.keras.layers.Input(shape=(299, 299, 3))

# x = inception_res_partial(input)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# x = tf.keras.layers.Dense(1024, activation='relu')(x)


# dig_data_input = tf.keras.layers.Input(shape=(3,))
# dig_data = tf.keras.layers.Dense(3, activation='relu')(dig_data_input)
# x = tf.keras.layers.Concatenate(axis=-1)([x,dig_data])
# x = tf.keras.layers.Dense(1027, activation='relu')(x)
# x = tf.keras.layers.Dense(3, activation='softmax')(x)

# dig_model = tf.keras.models.Model(inputs=[input,dig_data_input], outputs=x)

# dig_model.summary()


In [ ]:
class pretrained_model(tf.keras.Model):
    def __init__(self):
        super(pretrained_model, self).__init__()
        self.inception_res_model = tf.keras.applications.InceptionResNetV2(include_top=False,weights='imagenet',input_shape=(299, 299, 3))
        self.inception_res_partial = models.Model(inputs=self.inception_res_model.input, outputs=self.inception_res_model.output)
        self.global_average_pooling = tf.keras.layers.GlobalAveragePooling2D()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
    def call(self, input):

        x = self.inception_res_partial(input)
        x = self.global_average_pooling(x)
        x = self.dense1(x)
        return x

In [ ]:
class dignose_model(tf.keras.Model):
    def __init__(self):
        super(dignose_model, self).__init__()
        #### 특정 feature 가중치 ---> 젊은이에게 부정적 가중치
        ###### loss를 customize 해보자
        self.concat_layer = tf.keras.layers.Concatenate(axis=-1)
        self.reshape_layer = tf.keras.layers.Reshape((1,))  # 각 입력을 열 벡터로 변환하는 레이어 추가
        self.dense1 = tf.keras.layers.Dense(3, activation='relu')

    def call(self, inputs):
        gender, age_group, image_number = inputs
        # 각 입력을 열 벡터로 변환
        gender = self.reshape_layer(gender)
        age_group = self.reshape_layer(age_group)
        image_number = self.reshape_layer(image_number)

        x = self.concat_layer([gender, age_group, image_number])
        x = self.dense1(x)
        return x


In [ ]:
class concat_model(tf.keras.Model):
    def __init__(self):
        super(concat_model, self).__init__()
        self.pretrained_model = pretrained_model()
        self.dignose_model = dignose_model()
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        self.dense1 = tf.keras.layers.Dense(1027, activation='relu')
        self.dense2 = tf.keras.layers.Dense(2, activation='softmax')


    def call(self, inputs):
        img, diagnose= inputs
        img = self.pretrained_model(img)
        # print(img.shape)
        diagno = self.dignose_model(diagnose)
        # print(diagno.shape)
        x = self.concat((img,diagno))
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [ ]:
model = concat_model()

In [ ]:
for data, diagnose, labels in train_dataset_tensor.take(1):
    model([data,diagnose])
    break

In [ ]:
for data, diagnose, labels in train_dataset_tensor.take(1):
    print(data.shape)
    print(diagnose[0].shape)
    print(diagnose[1].shape)
    print(diagnose[2].shape)
    print(labels.shape)

(30, 299, 299, 3)
(30,)
(30,)
(30,)
(30,)


In [ ]:


model.summary()

Model: "concat_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pretrained_model_1 (pretra  multiple                  55910624  
 ined_model)                                                     
                                                                 
 dignose_model_1 (dignose_m  multiple                  12        
 odel)                                                           
                                                                 
 concatenate_3 (Concatenate  multiple                  0         
 )                                                               
                                                                 
 dense_6 (Dense)             multiple                  1055756   
                                                                 
 dense_7 (Dense)             multiple                  2056      
                                                    

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
if use_colab:
    checkpoint_dir ='/content/drive/MyDrive/아이펠/ai_트랙/train_ckpt/medical_baseline/exp1'
    if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)
else:
    checkpoint_dir = './train_ckpt/segmentation/exp1'

In [ ]:
if not tf.io.gfile.exists(checkpoint_dir):
    tf.io.gfile.makedirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

if is_train:
    checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                   model=model)
else:
    checkpoint = tf.train.Checkpoint(model=model)

In [ ]:
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=10)
cp_callback = ModelCheckpoint(
    checkpoint_dir, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:
def compute_loss(labels, predictions):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

In [ ]:
num_epochs = 100

In [ ]:
# 에폭 반복
for epoch in range(num_epochs):
    print("\nStart of epoch %d" % (epoch,))

    # 한 에폭당 평균 손실 초기화
    epoch_loss_avg = tf.keras.metrics.Mean()
    # validation 데이터셋에서 정확도를 평가하기 위한 지표 초기화
    val_accuracy = tf.keras.metrics.Accuracy()

    # 데이터셋 스텝 반복
    for step, (data, diagnose, labels) in enumerate(train_dataset_tensor):
        with tf.GradientTape() as tape:
            # 모델 호출
            predictions = model([data, diagnose], training=True)

            # 손실 계산
            loss = compute_loss(labels, predictions)

        # 기울기 계산
        gradients = tape.gradient(loss, model.trainable_variables)

        # 기울기를 적용하여 가중치 업데이트
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # 손실 기록
        epoch_loss_avg.update_state(loss)

        # 매 30스텝마다 평균 손실 출력 및 분류
        if (step + 1) % 30 == 0:
            mean_loss = epoch_loss_avg.result()
            print("Step {:03d}: Mean Loss: {:.3f}".format(step, mean_loss))

            # 분류 로직: 평균 손실을 기준으로 환자인지 아닌지를 분류
            if mean_loss > 0.5:
                print(f"mean_loss : {mean_loss: .2f} Mild AD입니다. 실제값 : {labels[0]}")
            else:
                print(f"mean_loss :{mean_loss} 정상입니다.실제값 :{labels[0]}")

    # validation 데이터셋을 사용하여 모델 평가
    for data, diagnose, labels in valid_dataset_tensor:
        predictions = model([data, diagnose], training=False)
        val_accuracy.update_state(labels, tf.argmax(predictions, axis=1))

    # validation 정확도 출력
    print("Validation Accuracy: {:.3f}".format(val_accuracy.result()))



Start of epoch 0
Step 029: Mean Loss: 0.694
mean_loss :  0.69 Mild AD입니다. 실제값 : 0
Step 059: Mean Loss: 0.684
mean_loss :  0.68 Mild AD입니다. 실제값 : 1
Step 089: Mean Loss: 0.695
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 119: Mean Loss: 0.695
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 149: Mean Loss: 0.697
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 179: Mean Loss: 0.697
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 209: Mean Loss: 0.697
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 239: Mean Loss: 0.696
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 269: Mean Loss: 0.695
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 299: Mean Loss: 0.695
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 329: Mean Loss: 0.696
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 359: Mean Loss: 0.695
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 389: Mean Loss: 0.695
mean_loss :  0.69 Mild AD입니다. 실제값 : 0
Validation Accuracy: 0.439

Start of epoch 1
Step 029: Mean Loss: 0.696
mean_loss :  0.70 Mild AD입니다. 실제값 : 0
Step 059: Mean Loss: 0.698


KeyboardInterrupt: 